In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from itertools import cycle, islice
from os import listdir, path
from os.path import isfile, join
from PIL import Image
import numpy as np
import math
from datetime import datetime

In [2]:
def red(image):
    arr = np.asarray(image).copy()
    rgb = arr[:,:,1:3]
    arr[:,:,1:3][np.where(rgb == 255)] = 0
    
    return Image.fromarray(arr)


def black(image):
    arr = np.asarray(image).copy()
    rgb = arr[:,:,0:3]
    arr[:,:,0:3][np.where(rgb == 255)] = 0
    
    return Image.fromarray(arr)


def load_rank_samples(rank_images_dir):
    for name in listdir(rank_images_dir):
        full_path = join(rank_images_dir, name)
        if not isfile(full_path):
            continue

        rank = path.splitext(path.basename(full_path))[0]
        image = Image.open(full_path)
        red_image = red(image)
        black_image = black(image)

        yield (red_image, rank)
        yield (black_image, rank)


def warp(img, angle, center, new_center, scale):
    angle = -angle/180.0*math.pi
    nx,ny = x,y = center
    sx=sy=1.0
    (nx,ny) = new_center
    (sx,sy) = scale
    cosine = math.cos(angle)
    sine = math.sin(angle)
    a = cosine/sx
    b = sine/sx
    c = x-nx*a-ny*b
    d = -sine/sy
    e = cosine/sy
    f = y-nx*d-ny*e
    
    fff = Image.new('RGBA', img.size, (255,)*4)
    warped_img = img.transform(img.size, Image.AFFINE, (a,b,c,d,e,f))
    img = Image.composite(warped_img, fff, warped_img)
    
    return img.convert('RGB')


def warp_rnd(image):
    r = lambda v: v * (.5 - np.random.rand())
    width, height = image.size
    
    angle = r(20)
    center = width/2 + r(width/10), height/2 + r(height/10)
    new_center = width/2 + r(width/20), height/2 + r(height/20)
    scale = 1 + r(.3), 1 + r(.3)
    
    return warp(image, angle, center, new_center, scale)


def gen_rank_samples(rank_images_dir, count):
    samples = list(load_rank_samples('res/cards/rank'))
    samples = [(warp_rnd(image), label) for (image, label) in islice(cycle(samples), 0, count)]
    images, labels = zip(*samples)
    return images, labels

In [3]:
start = datetime.now()
rank_images, rank_labels = gen_rank_samples('res/cards/rank', 100000)
finish = datetime.now()
print('Elapsed: %s' % (finish - start))

Elapsed: 0:00:05.491182


In [ ]:
def preprocess(image):
    warped_image = warp(image)
    return np.asarray(warped_image)

In [ ]:
def card_rank_model_fn(features, labels, mode, params, config):
    pass

def card_rank_train_input_fn(steps, batch_count, batch_size):
    (images, labels) = load_rank_images()
    
    stop = steps * batch_count * batch_size
    
    images = [preprocess(image) for image in islice(cycle(images), 0, stop)]
    labels = [label for label in islice(cycle(images), 0, stop)]
    
    return (images, labels)

def card_rank_eval_input_fn():
    pass

In [ ]:
classifier = tf.estimator.Estimator(
    model_fn = card_rank_model_fn,
    model_dir = 'models/card_rank'
)

train_steps = 1000
train_batch_count = 100
train_batch_size = 100

classifier.train(
    input_fn = lambda: card_rank_train_input_fn(train_steps, train_batch_count, train_batch_size),
    steps = train_steps
)

metrics = classifier.evaluate(
    input_fn = card_rank_eval_input_fn
)

print('Accuracy: {accuracy:0.3f}'.format(**metrics))